In [39]:
import os
import sys
os.chdir("/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/bin")
sys.path.append(os.getcwd()) 
import torch

from flower.task import Net, train
import torch
# import dataloader
from torch.utils.data import DataLoader
from lib.dataset_utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from lib.physionet import PhysioNet, variable_time_collate_fn, get_data_min_max
from sklearn import model_selection
from torch.utils.data import DataLoader
from lib import utils
from lib.dataset_utils import store_dataset_physionet

path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet"
batch_size = 64
classif = False
noise_weight = 0.1
max_t_extrap = 5
n_total_tp = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
amp_start = 1.0
amp_end = 1.0 
freq_start =1.0
freq_end = freq_start
epochs = 5
batch_size = 3
sample_tp = 0.9
n_samples = 4
max_t_extrap = 50
lr_val = 0.01
cut_tp = None
extrap = None

from types import SimpleNamespace
args = SimpleNamespace()
args.sample_tp = 0
args.cut_tp = None
args.extrap = None


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:

#################################################################
# Physionet Dataset - FETCH & FORMAT
#################################################################
quantization = 0.1 # quantization of time stamps in 0.1 hours
n = 10000 # number of samples to use from the dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_dataset_obj = PhysioNet('../data/physionet', train=True, 
                                quantization = quantization,
                                download=False, n_samples = min(10000, n), 
                                device = device)
# Use custom collate_fn to combine samples with arbitrary time observations.
# Returns the dataset along with mask and time steps
test_dataset_obj = PhysioNet('../data/physionet', train=False, 
                                quantization = quantization,
                                download=False, n_samples = min(10000,n), 
                                device = device)

# Combine and shuffle samples from physionet Train and physionet Test
total_dataset = train_dataset_obj[:len(train_dataset_obj)]
attr_names = train_dataset_obj.params

## Store full dataset

In [41]:
data_min, data_max = get_data_min_max(total_dataset)

client_0, client_1 = model_selection.train_test_split(total_dataset, test_size=0.5, random_state=42)
train_data_0, test_data_0 = model_selection.train_test_split(client_0, train_size= 0.8, 
    random_state = 42, shuffle = True)
train_data_1, test_data_1 = model_selection.train_test_split(client_1, train_size= 0.8,
    random_state = 42, shuffle = True)

#################################################################
# Physionet Dataset - STORE
#################################################################
path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet"


store_dataset_physionet(train_data_0, test_data_0, dataset_prefix="physionet", path_prefix=path_prefix, client_prefix=f"client_0")
store_dataset_physionet(train_data_1, test_data_1, dataset_prefix="physionet", path_prefix=path_prefix, client_prefix=f"client_1")
torch.save(attr_names, f"{path_prefix}/{prefix}/attr_names.pt")
torch.save(data_min, f"{path_prefix}/{prefix}/client_0_data_min.pt")
torch.save(data_max, f"{path_prefix}/{prefix}/client_0_data_max.pt")
torch.save(data_min, f"{path_prefix}/{prefix}/client_1_data_min.pt")
torch.save(data_max, f"{path_prefix}/{prefix}/client_1_data_max.pt")

In [43]:
train_data_0[1][2].shape

torch.Size([92, 41])

-----------------------------------

## Create a subset dataset, where we only keep the features that are more continous


In [36]:
attr_names = torch.load(f"{path_prefix}/{prefix}/attr_names.pt", weights_only=False)
threshold_attribute = 10
attr_names = np.array(attr_names)
mask_sum = np.array([x[3].sum(axis=0) for x in total_dataset])
attribute_filtered_idx = mask_sum.mean(axis=0) > 10
attr_names_filtered = attr_names[attribute_filtered_idx]
index_filtered = np.where(attribute_filtered_idx)[0]
print(f"Filtered attributes: {attr_names_filtered}")
print(f"Number of attributes after filtering: {len(attr_names_filtered)}")

Filtered attributes: ['Weight' 'DiasABP' 'GCS' 'HR' 'MAP' 'NIDiasABP' 'NIMAP' 'NISysABP'
 'RespRate' 'SysABP' 'Temp' 'Urine']
Number of attributes after filtering: 12


In [37]:
# now downsample the dataset to only include attributes that are above the threshold
filtered_dataset = []
for sample in total_dataset:
    # use  index_filtered to filter the sampl
    filtered_sample = (sample[0], sample[1], sample[2][:,index_filtered], sample[3][:, index_filtered], sample[4])
    filtered_dataset.append(filtered_sample)

In [38]:
data_min, data_max = get_data_min_max(filtered_dataset)

client_0, client_1 = model_selection.train_test_split(filtered_dataset, test_size=0.5, random_state=42)
train_data_0, test_data_0 = model_selection.train_test_split(client_0, train_size= 0.8, 
    random_state = 42, shuffle = True)
train_data_1, test_data_1 = model_selection.train_test_split(client_1, train_size= 0.8,
    random_state = 42, shuffle = True)

#################################################################
# Physionet Dataset - STORE
#################################################################
path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet_filtered"


store_dataset_physionet(train_data_0, test_data_0, dataset_prefix="physionet_filtered", path_prefix=path_prefix, client_prefix=f"client_0")
store_dataset_physionet(train_data_1, test_data_1, dataset_prefix="physionet_filtered", path_prefix=path_prefix, client_prefix=f"client_1")
torch.save(attr_names, f"{path_prefix}/{prefix}/attr_names.pt")
torch.save(data_min, f"{path_prefix}/{prefix}/client_0_data_min.pt")
torch.save(data_max, f"{path_prefix}/{prefix}/client_0_data_max.pt")
torch.save(data_min, f"{path_prefix}/{prefix}/client_1_data_min.pt")
torch.save(data_max, f"{path_prefix}/{prefix}/client_1_data_max.pt")